# Python helper file/script for MATLAB

This cell sets up the notebook to import numpy, datascience, seaborn, pandas, matplotlib etc.

In [27]:
# Run this cell to set up the notebook.

# These lines import the Numpy, Datascience, pandas modules.
import numpy as np
import pandas as pd
import seaborn as sns
#from datascience import *
import random
import matplotlib
import matplotlib.pyplot as plt

# Importing plotting libraries and styles
%matplotlib inline
plt.style.use('fivethirtyeight')

# For Pandas to ignore FutureWarning displays
import warnings
warnings.simplefilter('ignore', FutureWarning)

### The cell given below sets up MATLAB for the notebook
Source: https://sehyoun.com/blog/20180904_using-matlab-with-jupyter-notebook.html

In [66]:
import matlab.engine
import io
import scipy.io
from IPython.core.magic import register_cell_magic
ip = get_ipython()

out = io.StringIO()
err = io.StringIO()

# Setup matlab cell magic #
@register_cell_magic
def matlab_magic(line,cell):
    out.truncate(0)
    out.seek(0)
    err.truncate(0)
    err.truncate(0)
    raw = '''{line}.eval("""{cell}""", nargout=0, stdout=out, stderr=err)'''
    ip.run_cell(raw.format(line=line, cell=cell))
    print(out.getvalue())
    print(err.getvalue())
    
# Starting a MATLAB engine called eng
eng = matlab.engine.start_matlab()

In [67]:
%%matlab_magic eng

t = linspace(0,6*pi,100);
plot(sin(t))
grid on
hold on
plot(cos(t), 'r')

In [68]:
%%matlab_magic eng

x = 5;
disp(x);
x

     5


x =

     5





Calling another MATLAB file from Python script

In [69]:
%%matlab_magic eng

my_array = [10, 20, 30]
avgOfThis = averageFunc(my_array)


my_array =

    10    20    30


avgOfThis =

    20





**Python to MATLAB**

Source: https://stackoverflow.com/questions/10997254/converting-numpy-arrays-to-matlab-and-vice-versa

In [70]:
x = [1,2,3]
y = "hello"
z = [[1,2,3], [4,5,6]]

scipy.io.savemat('test.mat', dict(x=x, y=y, z=z))

In [74]:
%%matlab_magic eng
load test
x(1)
y
z


ans =

  int64

   1


y =

    'hello'


z =

  2x3 int64 matrix

   1   2   3
   4   5   6





**MATLAB to Python**

Source: http://www.blogforbrains.com/blog/2014/9/6/loading-matlab-mat-data-in-python

In [37]:
# Similarly we have
#scipy.io.loadmat("averageFunc.m")
variables = scipy.io.loadmat('data.mat', squeeze_me=True)
variables

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Fri Feb  8 01:20:47 2019',
 '__version__': '1.0',
 '__globals__': [],
 'a': array([[1, 2, 3],
        [4, 5, 6]], dtype=uint8),
 'S': array((array([[ 7,  8,  9],
        [10, 11, 12]], dtype=uint8),), dtype=[('b', 'O')]),
 'M': array([(array([[ 2,  4,  6],
        [ 8, 10, 12]], dtype=uint8),),
        (array([[ 1,  3,  5],
        [ 7,  9, 11]], dtype=uint8),)], dtype=[('c', 'O')])}

In [38]:
a = variables['a'] # array
S = variables['S'] # structure containing an array
M = variables['M'] # array of structures

a

array([[1, 2, 3],
       [4, 5, 6]], dtype=uint8)

In [39]:
S

array((array([[ 7,  8,  9],
       [10, 11, 12]], dtype=uint8),), dtype=[('b', 'O')])

In [40]:
M

array([(array([[ 2,  4,  6],
       [ 8, 10, 12]], dtype=uint8),),
       (array([[ 1,  3,  5],
       [ 7,  9, 11]], dtype=uint8),)], dtype=[('c', 'O')])

In [41]:
# Adds the MMB.m as well as MMBOPT1.m and MMBOPT2.m folders to the MATLAB engine path"
eng.addpath(r'/Users/Desktop/monetaryPolicy/mmb-gui-mlab/2.3.2', nargout=0)
eng.addpath(r'/Users/Desktop/monetaryPolicy/mmb-gui-mlab/2.3.2/MMB_OPTIONS', nargout=0)

# Important:
The code below sets the coefficients and other data for the PID rule to work.

In [107]:
# This sets the coefficients of the monetary policy rule, there are 33 coefficients and len(coefficients) = 33
coefficients = [0, 0, 0, 0, 1.5/4, 1.5/4, 1.5/4, 1.5/4, 
                0, 0, 0, 0, 0, 0, 0, 0, 
                0, 0, 0, 0, 0, 0, 0, 0, 
                0, 0, 0, 0, 0, 0, 0, 1, 0.25]

# Number of the model you want to chooose, please exclude 69-79, 19-22, 27, 59, 65, 68, 81, 97, 98
start = 1
end = 1


scipy.io.savemat('variables.mat', dict(coefficients=coefficients, modelStart = start, modelEnd=end))

## **Important:** 
The cell below runs the MMB.m file

In [112]:
eng.MMB(nargout = 0)

MatlabExecutionError: 
  File /Users/rishabsrivastava/Desktop/monetaryPolicy/mmb-gui-mlab-2.3.2/MMB.m, line 15, in MMB
'../scripts/variables.mat' is not found in the current folder or on the MATLAB path, but exists in:
    /Users/rishabsrivastava/Desktop/monetaryPolicy/mmb-gui-mlab-2.3.2

Change the MATLAB current folder or add its folder to the MATLAB path.


### Functions defined to import data for:
* 4 IRF: Impulse Response Function Variables (outputgap, inflation, interest, output)
* All IRF Variables
* 4 ACF: Autocorrelation Function Variables (outputgap, inflation, interest, output)


In [103]:
def singleModel_irf4():
    irf_4 = pd.read_excel("../mmb-gui-mlab-2.3.2/OUTPUT/results.xls", sheetname = "IRF Mon. Pol. Shock      ")
    irf_4 = irf_4.T
    irf_headers = irf_4.iloc[0] # grab the first row for the header
    irf_4 = irf_4[1:] # take the data less the header row
    irf_4_stripped_headers = [myHeader.strip() for myHeader in np.array(irf_headers)] # removing trailing whitespaces
    irf_4.columns = irf_4_stripped_headers
    modelName = irf_4.columns.values[1]
    irf_4 = irf_4.iloc[:, [i for i in range(1, len(irf_4.columns.values), 2)]]
    irf_4.columns = ["OutputGap", "Inflation", "Interest", "Output"]
    irf_4 = irf_4.reset_index()
    irf_4.index.name = "Period"
    irf_4.drop('index', axis=1, inplace=True)
    return irf_4, modelName

def singleModel_allirf():
    old_irf_df = pd.read_excel("../mmb-gui-mlab-2.3.2/OUTPUT/results.xls", sheetname = "all IRFs Mon. Pol. Shock")
    all_irf = old_irf_df.T
    new_header = all_irf.iloc[0] # grab the first row for the header
    all_irf = all_irf[1:] # take the data less the header row
    stripped_headers = [myHeader.strip() for myHeader in np.array(new_header)] # removing trailing whitespaces
    all_irf.columns = stripped_headers # set the header row as the df header
    all_irf["c_t"] = all_irf.index
    all_irf.index = np.arange(0,21,1)
    all_irf.index.name = "Period"

    # This section rearranges the columns
    n = len(list(all_irf.columns.values))
    rearranged = [list(all_irf.columns.values)[-1]] + list(all_irf.columns.values)[:n-1]
    all_irf = all_irf[rearranged]
    return all_irf

def singleModel_acf():
    acf = pd.read_excel("../mmb-gui-mlab-2.3.2/OUTPUT/results.xls", sheetname = "ACF")
    acf = acf.T
    acf_headers = acf.iloc[0] # grab the first row for the header
    acf = acf[1:] # take the data less the header row
    acf_stripped_headers = [myHeader.strip() for myHeader in np.array(acf_headers)] # removing trailing whitespaces
    acf.columns = acf_stripped_headers
    acf = acf.iloc[:, [i for i in range(0, len(acf.columns.values), 2)]]
    acf.columns = ["OutputGap", "Inflation", "Interest", "Output"]
    acf = acf.reset_index()
    acf.index.name = "Period"
    acf.drop('index', axis=1, inplace=True)
    return acf

In [104]:
singleModel_acf()

,OutputGap,Inflation,Interest,Output
Period,,,,
0,1,1,1,1
1,0.536911,0.754082,0.754082,0.610285
2,0.248427,0.558851,0.558851,0.367156
3,0.0770577,0.368061,0.368061,0.219724
4,0.00137064,0.143743,0.143743,0.143081
5,0.0333863,0.15455,0.15455,0.133783
6,0.0319908,0.135618,0.135618,0.109869
7,0.0235166,0.108023,0.108023,0.0865773
8,0.0173222,0.0835915,0.0835915,0.068419
